In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8051"

In [4]:
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

def research_messages_hook(payload):
    connection_id = payload["connection_id"]
    print("Handle research messages ", payload, connection_id)

research_message_listener = {
    "handler": research_messages_hook,
    "topic": "basicmessages"
}



In [5]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py:42>>

In [6]:
agent_controller.register_listeners([research_message_listener], defaults=True)

In [7]:
#Paste in invitation from data_owner agent
invite = {'connection_id': '366aa320-3e6b-435c-8415-cdd8b643617a', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '1ddbe7f4-89e3-435f-bb50-946b079b94af', 'label': 'Bob', 'serviceEndpoint': 'http://172.17.0.1:8020', 'recipientKeys': ['GMFbr3hc2Phj9D9YW4JWTaAd3n1itNUxaLjHgVGPBYu5']}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWRkYmU3ZjQtODllMy00MzVmLWJiNTAtOTQ2YjA3OWI5NGFmIiwgImxhYmVsIjogIkJvYiIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE3Mi4xNy4wLjE6ODAyMCIsICJyZWNpcGllbnRLZXlzIjogWyJHTUZicjNoYzJQaGo5RDlZVzRKV1RhQWQzbjFpdE5VeGFMakhnVkdQQll1NSJdfQ=='}



In [8]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
print(response)
# Print out accepted Invite and Researcher ID
print("Connection", response)
researcher_id = response["connection_id"]


ACCEPTED CONNECTION -  {'my_did': 'GCWXyezyt8WMjtVBcp3qoJ', 'routing_state': 'none', 'updated_at': '2020-06-25 15:36:28.336999Z', 'initiator': 'external', 'invitation_key': 'GMFbr3hc2Phj9D9YW4JWTaAd3n1itNUxaLjHgVGPBYu5', 'their_label': 'Bob', 'created_at': '2020-06-25 15:36:28.320580Z', 'connection_id': '457cb26e-364a-446f-a78c-0c339fcdfae5', 'request_id': '729b6466-c860-439c-95b9-9f694bfc051b', 'state': 'request', 'invitation_mode': 'once', 'accept': 'manual'}
{'my_did': 'GCWXyezyt8WMjtVBcp3qoJ', 'routing_state': 'none', 'updated_at': '2020-06-25 15:36:28.336999Z', 'initiator': 'external', 'invitation_key': 'GMFbr3hc2Phj9D9YW4JWTaAd3n1itNUxaLjHgVGPBYu5', 'their_label': 'Bob', 'created_at': '2020-06-25 15:36:28.320580Z', 'connection_id': '457cb26e-364a-446f-a78c-0c339fcdfae5', 'request_id': '729b6466-c860-439c-95b9-9f694bfc051b', 'state': 'request', 'invitation_mode': 'once', 'accept': 'manual'}
Connection {'my_did': 'GCWXyezyt8WMjtVBcp3qoJ', 'routing_state': 'none', 'updated_at': '202

In [9]:
# Do a trust ping to ensure connection is accepted by data_owner
trust_ping = await agent_controller.connections.trust_ping(researcher_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA")
print(trust_ping)   

TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA
{'thread_id': 'f77cfe69-175a-4611-9726-2010eb28cbeb'}


In [10]:
# Print connection list
connection = await agent_controller.connections.get_connection(researcher_id)
print("RESEARCH AGENT CONNECTION")
print(connection)

RESEARCH AGENT CONNECTION
{'my_did': 'GCWXyezyt8WMjtVBcp3qoJ', 'routing_state': 'none', 'updated_at': '2020-06-25 15:36:52.460947Z', 'initiator': 'external', 'invitation_key': 'GMFbr3hc2Phj9D9YW4JWTaAd3n1itNUxaLjHgVGPBYu5', 'their_did': 'StD1yXEWmmqSyKPZPdRrAQ', 'their_label': 'Bob', 'created_at': '2020-06-25 15:36:28.320580Z', 'connection_id': '457cb26e-364a-446f-a78c-0c339fcdfae5', 'request_id': '729b6466-c860-439c-95b9-9f694bfc051b', 'state': 'active', 'invitation_mode': 'once', 'accept': 'manual'}


In [11]:
#send some basic messages
message = await agent_controller.connections.send_message(researcher_id,"hello from researcher world!")
print("BASIC MESSAGE - RESEARCH -> DATA")
print(message)

BASIC MESSAGE - RESEARCH -> DATA
{}
